# Module 7: Unsupervised Learning

---

Unsupervised learning discovers hidden patterns in data **without labeled outputs**. This module covers the two major categories: **clustering** (grouping similar data points) and **dimensionality reduction** (compressing high-dimensional data into fewer dimensions while preserving structure).

---

## Table of Contents

1. [K-Means Clustering](#1.-K-Means-Clustering)
2. [Hierarchical Clustering](#2.-Hierarchical-Clustering)
3. [DBSCAN](#3.-DBSCAN)
4. [Dimensionality Reduction: PCA](#4.-Dimensionality-Reduction:-PCA)
5. [Visualization with t-SNE](#5.-Visualization-with-t-SNE)
6. [Exercises](#6.-Exercises)
7. [Summary and Further Reading](#7.-Summary-and-Further-Reading)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_blobs, make_moons, load_iris, load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, silhouette_samples
from scipy.cluster.hierarchy import dendrogram, linkage

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

---

## 1. K-Means Clustering

K-Means partitions data into **K clusters** by iteratively:
1. Assigning each point to the nearest centroid.
2. Updating centroids as the mean of all assigned points.
3. Repeating until convergence.

The key challenge: you must choose K in advance.

In [ ]:
# Generate synthetic data with 4 clusters
X_blobs, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.8, random_state=42)

# Apply K-Means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
y_kmeans = kmeans.fit_predict(X_blobs)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Before clustering (unlabeled)
axes[0].scatter(X_blobs[:, 0], X_blobs[:, 1], c='gray', s=30, alpha=0.6, edgecolors='white', linewidth=0.3)
axes[0].set_title('Raw Data (No Labels)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')

# After clustering
scatter = axes[1].scatter(X_blobs[:, 0], X_blobs[:, 1], c=y_kmeans, cmap='viridis',
                          s=30, alpha=0.7, edgecolors='white', linewidth=0.3)
axes[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
                c='red', marker='X', s=200, edgecolors='black', linewidth=1, label='Centroids')
axes[1].set_title('K-Means Clustering (K=4)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].legend(fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
# The Elbow Method: choosing the optimal K
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_blobs)
    inertias.append(km.inertia_)
    silhouette_scores.append(silhouette_score(X_blobs, km.labels_))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elbow plot
axes[0].plot(K_range, inertias, 'o-', linewidth=2, color='#2196F3', markersize=8)
axes[0].axvline(x=4, color='red', linestyle='--', alpha=0.7, label='Elbow at K=4')
axes[0].set_xlabel('Number of Clusters (K)', fontsize=13)
axes[0].set_ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=13)
axes[0].set_title('Elbow Method', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)

# Silhouette score
axes[1].plot(K_range, silhouette_scores, 's-', linewidth=2, color='#FF5722', markersize=8)
axes[1].axvline(x=4, color='red', linestyle='--', alpha=0.7, label='Best K=4')
axes[1].set_xlabel('Number of Clusters (K)', fontsize=13)
axes[1].set_ylabel('Silhouette Score', fontsize=13)
axes[1].set_title('Silhouette Score', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)

plt.tight_layout()
plt.show()

print("The Elbow Method: look for the 'bend' where adding more clusters gives diminishing returns.")
print("Silhouette Score: ranges from -1 to 1. Higher is better (well-separated clusters).")

---

## 2. Hierarchical Clustering

Hierarchical clustering builds a tree (dendrogram) of cluster merges. It does not require specifying K in advance — you can cut the tree at any level to get the desired number of clusters.

**Agglomerative (bottom-up)**: Start with each point as its own cluster, then merge the closest pairs.

In [ ]:
# Dendrogram
X_small = X_blobs[:50]  # use a subset for readable dendrogram

linkage_matrix = linkage(X_small, method='ward')

fig, ax = plt.subplots(figsize=(16, 6))
dendrogram(linkage_matrix, ax=ax, leaf_rotation=90, leaf_font_size=8,
           color_threshold=5)
ax.set_title('Hierarchical Clustering Dendrogram', fontsize=14, fontweight='bold')
ax.set_xlabel('Sample Index', fontsize=13)
ax.set_ylabel('Distance', fontsize=13)
ax.axhline(y=5, color='red', linestyle='--', alpha=0.7, label='Cut threshold')
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

print("The dendrogram shows how clusters are merged at each step.")
print("Cutting at a specific height determines the number of clusters.")

In [ ]:
# Agglomerative Clustering on the full dataset
agg = AgglomerativeClustering(n_clusters=4, linkage='ward')
y_agg = agg.fit_predict(X_blobs)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(X_blobs[:, 0], X_blobs[:, 1], c=y_kmeans, cmap='viridis',
                s=30, alpha=0.7, edgecolors='white', linewidth=0.3)
axes[0].set_title('K-Means (K=4)', fontsize=14, fontweight='bold')

axes[1].scatter(X_blobs[:, 0], X_blobs[:, 1], c=y_agg, cmap='viridis',
                s=30, alpha=0.7, edgecolors='white', linewidth=0.3)
axes[1].set_title('Agglomerative Clustering (K=4)', fontsize=14, fontweight='bold')

for ax in axes:
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')

plt.suptitle('K-Means vs Hierarchical Clustering', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 3. DBSCAN

**DBSCAN (Density-Based Spatial Clustering of Applications with Noise)** finds arbitrarily shaped clusters based on density. Unlike K-Means, it:
- Does not require specifying the number of clusters
- Can identify noise points (outliers)
- Handles non-spherical cluster shapes

Key parameters:
- **eps**: Maximum distance between two points to be considered neighbors
- **min_samples**: Minimum number of points to form a dense region

In [ ]:
# Generate non-spherical data (Moons)
X_moons, y_moons = make_moons(n_samples=300, noise=0.1, random_state=42)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# K-Means (fails on non-spherical shapes)
km_moons = KMeans(n_clusters=2, random_state=42, n_init=10)
y_km_moons = km_moons.fit_predict(X_moons)
axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c=y_km_moons, cmap='viridis',
                s=30, edgecolors='white', linewidth=0.3)
axes[0].set_title('K-Means (K=2)', fontsize=13, fontweight='bold')

# DBSCAN (handles non-spherical shapes)
dbscan = DBSCAN(eps=0.2, min_samples=5)
y_db = dbscan.fit_predict(X_moons)
axes[1].scatter(X_moons[:, 0], X_moons[:, 1], c=y_db, cmap='viridis',
                s=30, edgecolors='white', linewidth=0.3)
n_noise = (y_db == -1).sum()
axes[1].set_title(f'DBSCAN (eps=0.2)\nNoise points: {n_noise}', fontsize=13, fontweight='bold')

# True labels
axes[2].scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='viridis',
                s=30, edgecolors='white', linewidth=0.3)
axes[2].set_title('True Labels', fontsize=13, fontweight='bold')

for ax in axes:
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')

plt.suptitle('DBSCAN Handles Non-Spherical Clusters', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("K-Means assumes spherical clusters and fails on the Moons data.")
print("DBSCAN correctly identifies the two interleaved half-circles.")

---

## 4. Dimensionality Reduction: PCA

**Principal Component Analysis (PCA)** reduces the number of features by projecting data onto the directions of maximum variance. Uses:
- Visualization of high-dimensional data
- Noise reduction
- Speeding up ML algorithms
- Feature extraction

In [ ]:
# PCA on the Iris dataset
iris = load_iris()
X_iris = StandardScaler().fit_transform(iris.data)

# Reduce from 4D to 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_iris)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PCA projection
for i, name in enumerate(iris.target_names):
    mask = iris.target == i
    axes[0].scatter(X_pca[mask, 0], X_pca[mask, 1], label=name, s=40,
                    edgecolors='white', linewidth=0.3, alpha=0.7)
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
axes[0].set_title('Iris Dataset — PCA (4D to 2D)', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)

# Explained variance
pca_full = PCA().fit(X_iris)
cumulative_var = np.cumsum(pca_full.explained_variance_ratio_)
axes[1].bar(range(1, 5), pca_full.explained_variance_ratio_, color='#2196F3',
            edgecolor='white', alpha=0.7, label='Individual')
axes[1].plot(range(1, 5), cumulative_var, 'ro-', linewidth=2, markersize=8, label='Cumulative')
axes[1].axhline(y=0.95, color='gray', linestyle='--', alpha=0.5, label='95% threshold')
axes[1].set_xlabel('Principal Component', fontsize=12)
axes[1].set_ylabel('Explained Variance Ratio', fontsize=12)
axes[1].set_title('Explained Variance by Component', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(1, 5))
axes[1].legend(fontsize=10)

plt.tight_layout()
plt.show()

print(f"Explained variance by component: {pca_full.explained_variance_ratio_.round(3)}")
print(f"Cumulative: {cumulative_var.round(3)}")
print(f"\nThe first 2 components capture {cumulative_var[1]:.1%} of the total variance.")

In [ ]:
# PCA on a higher-dimensional dataset (Digits)
digits = load_digits()
X_digits = StandardScaler().fit_transform(digits.data)

print(f"Digits dataset: {digits.data.shape[0]} samples, {digits.data.shape[1]} features")
print(f"Each sample is an 8x8 pixel image of a handwritten digit (0-9)")

# Reduce 64D to 2D for visualization
pca_digits = PCA(n_components=2)
X_digits_2d = pca_digits.fit_transform(X_digits)

fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(X_digits_2d[:, 0], X_digits_2d[:, 1], c=digits.target,
                     cmap='tab10', s=10, alpha=0.6)
plt.colorbar(scatter, ax=ax, label='Digit')
ax.set_xlabel(f'PC1 ({pca_digits.explained_variance_ratio_[0]:.1%})', fontsize=13)
ax.set_ylabel(f'PC2 ({pca_digits.explained_variance_ratio_[1]:.1%})', fontsize=13)
ax.set_title('Digits Dataset — PCA (64D to 2D)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 5. Visualization with t-SNE

**t-SNE (t-distributed Stochastic Neighbor Embedding)** is a nonlinear dimensionality reduction technique designed specifically for **visualization**. It preserves local structure, making clusters more visually apparent.

Key differences from PCA:
- PCA is linear; t-SNE is nonlinear
- PCA preserves global variance; t-SNE preserves local neighborhoods
- t-SNE is used for visualization only (not for feature extraction)

In [ ]:
# t-SNE on Digits dataset
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_digits_tsne = tsne.fit_transform(X_digits)

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# PCA
scatter1 = axes[0].scatter(X_digits_2d[:, 0], X_digits_2d[:, 1], c=digits.target,
                           cmap='tab10', s=10, alpha=0.6)
axes[0].set_title('PCA (64D to 2D)', fontsize=14, fontweight='bold')

# t-SNE
scatter2 = axes[1].scatter(X_digits_tsne[:, 0], X_digits_tsne[:, 1], c=digits.target,
                           cmap='tab10', s=10, alpha=0.6)
axes[1].set_title('t-SNE (64D to 2D)', fontsize=14, fontweight='bold')

for ax in axes:
    ax.set_xlabel('Component 1')
    ax.set_ylabel('Component 2')

plt.colorbar(scatter2, ax=axes[1], label='Digit')
plt.suptitle('PCA vs t-SNE on Handwritten Digits', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("t-SNE produces much better-separated clusters for visualization purposes.")
print("However, the axes have no interpretable meaning (unlike PCA components).")

---

## 6. Exercises

### Exercise 1: Customer Segmentation

In [ ]:
# Exercise 1: Segment synthetic customer data
#
# 1. Generate customer data with make_blobs (n_samples=500, centers=5)
# 2. Use the Elbow Method to determine the optimal K
# 3. Apply K-Means with the optimal K
# 4. Compute and print the silhouette score
# 5. Visualize the clusters with centroids

# Your code here:


### Exercise 2: DBSCAN Parameter Sensitivity

In [ ]:
# Exercise 2: Explore DBSCAN's sensitivity to eps parameter
#
# Using the make_moons data created earlier:
# 1. Try eps values: [0.05, 0.1, 0.2, 0.3, 0.5]
# 2. For each, run DBSCAN and record: number of clusters, number of noise points
# 3. Create a 1x5 subplot showing the clustering result for each eps
# 4. Which eps value gives the best result? Why?

# Your code here:


### Exercise 3: PCA for Model Speedup

In [ ]:
# Exercise 3: Test whether PCA can speed up training without losing accuracy
#
# Using the Digits dataset:
# 1. Train a KNN classifier (K=5) on the original 64D data and measure accuracy + time
# 2. Use PCA to reduce to 20 dimensions, then train KNN and measure accuracy + time
# 3. Try 10 dimensions as well
# 4. Compare accuracy and training time
# 5. What is the minimum number of PCA components that retains > 95% accuracy?
#
# Hint: use time.time() or %%timeit to measure execution time

import time
from sklearn.neighbors import KNeighborsClassifier

# Your code here:


---

## 7. Summary and Further Reading

### What We Covered

| Algorithm | Type | Key Feature | Limitation |
|-----------|------|-------------|------------|
| K-Means | Clustering | Fast, simple | Must specify K, assumes spherical clusters |
| Hierarchical | Clustering | Dendrogram visualization | Slow on large datasets |
| DBSCAN | Clustering | Finds arbitrary shapes, detects noise | Sensitive to eps and min_samples |
| PCA | Dim. Reduction | Linear, interpretable, fast | Only captures linear relationships |
| t-SNE | Visualization | Excellent cluster separation in 2D | Slow, non-deterministic, visualization only |

### Recommended Reading

- [Scikit-learn Clustering Guide](https://scikit-learn.org/stable/modules/clustering.html)
- [Scikit-learn Decomposition Guide](https://scikit-learn.org/stable/modules/decomposition.html)
- Chapter 8 of Aurélien Géron, *Hands-On Machine Learning* (Dimensionality Reduction)
- Chapter 9 of Aurélien Géron, *Hands-On Machine Learning* (Unsupervised Learning)

### Next Module

In **Module 8: Ensemble Methods**, we will learn how combining multiple models (bagging, boosting, stacking) can dramatically improve predictive performance.

---